In [4]:
%load_ext autoreload
%autoreload 2

from typing import List, Dict
import langfun as lf

from radar.evaluate.baselines import direct_prompt
from radar.data import load_task_instances_hf

tasks, df_stats = load_task_instances_hf(split="tasks")
print(f"Number of tasks: {len(tasks)}")
df_stats.head(2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Number of tasks: 313


,task_id,query,answer,artifact_type,artifact_scope,query_cols,artifact_reasoning_cols,table,num_rows,num_cols,recovered_tables_transform_spec,base_data_num_tokens,base_data_token_bucket,perturbation_note
0,ultra-trail-races-rank,We have a dataset of ultra trail running race ...,40.48,outliers,single-column,"[race_year_id, rank, age]",[age],"{'headers': ['race_year_id', 'race', 'rank', '...",113,10,"{'drop_rows': [[]], 'overwrite_cells': [[]]}",7995,8000,Introduced a obvious outliers in age column. S...
1,ultra-trail-races-rank,We have a dataset of ultra trail running race ...,40.13,outliers,single-column,"[race_year_id, rank, age]",[rank],"{'headers': ['race_year_id', 'race', 'rank', '...",113,10,"{'drop_rows': [[]], 'overwrite_cells': [[{'row...",7995,8000,Introduced formatting inconsistencies in rank ...


In [5]:
def llm_func(prompt_messages: List[Dict[str, str]]):
    # TODO: add own API key and model
    llm = lf.LanguageModel.get("gpt-4o-mini",
                               debug=True)
    lf_messages = [lf.Message.from_value(m, format="openai") for m in prompt_messages]
    response: lf.Message = lf.query(lf_messages, lm=llm, returns_message=True)
    
    return response.text, response.sym_jsonify()["metadata"]

task_res = direct_prompt.run_direct_prompt(tasks[0], llm_func)
task_res

UnboundLocalError: cannot access local variable 'answer' where it is not associated with a value

In [5]:
task_res.model_dump()

{'baseline': 'direct_prompt',
 'task': {'task_id': 'ultra-trail-races-rank',
  'query': 'We have a dataset of ultra trail running race results.\nWhat is the average age of all top 5 finishers (i.e., any result with a rank of 1-5) in this dataset?\nReturn the result rounded to 2 decimal places.',
  'artifact_type': 'outliers',
  'artifact_scope': 'single-column',
  'query_cols': ['race_year_id', 'rank', 'age'],
  'artifact_reasoning_cols': ['age'],
  'table': {'headers': ['race_year_id',
    'race',
    'rank',
    'runner',
    'age',
    'time',
    'time_in_seconds',
    'date',
    'event',
    'distance'],
   'rows': [['68140',
     'Millstone 100',
     '1.0',
     'VERHEUL Jasper',
     '30',
     '26H 35M 25S',
     '95725.0',
     '2021-09-03',
     'Peak District Ultras',
     '166.9'],
    ['68140',
     'Millstone 100',
     '2.0',
     'MOULDING JON',
     '43',
     '27H 0M 29S',
     '97229.0',
     '2021-09-03',
     'Peak District Ultras',
     '166.9'],
    ['68140',
 